In [1]:
import sys
!{sys.executable} -m pip install opencv-contrib-python --upgrade
!{sys.executable} -m pip install colorthief

In [2]:
import cv2
import numpy as np
import math
from colorthief import ColorThief
import time

# Read the image
image = cv2.imread("./data/side.jpg")

cv2.imshow("Original", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
def get_swatch_from_img(img, txt):
    vertex = []
    copy = img.copy()

    def swatch(e, x, y, f, param):
        if e == cv2.EVENT_LBUTTONDOWN:
            vertex.append((x, y))
            if len(vertex) % 2 == 0:
                cv2.rectangle(copy, vertex[-2], vertex[-1], (0, 255, 0), 1)

    cv2.namedWindow(txt)
    cv2.setMouseCallback(txt, swatch)

    while True:
        cv2.imshow(txt, copy)
        key = cv2.waitKey(1)
        if key != -1:
            break
    cv2.destroyWindow(txt)
    return vertex


def get_swatch(img, txt):
    swatch = get_swatch_from_img(img, txt)
    x, y, x_f, y_f = swatch[0][0], swatch[0][1], swatch[1][0], swatch[1][1]
    (x, x_f) = (x_f, x) if x > x_f else (x, x_f)
    (y, y_f) = (y_f, y) if y > y_f else (y, y_f)

    return img[y:y_f, x:x_f, :]


In [4]:
def dominant_color(img):
    cv2.imwrite("./data/dominant.jpg", img)
    time.sleep(10)
    color_thief = ColorThief("./data/dominant.jpg")
    color = color_thief.get_color(quality=1)
    return [color[2], color[1], color[0]]


def get_binary_mask(channel, shape, c_min, c_max):
    channel = np.resize(channel, shape[:2])
    mask = (channel >= c_min) & (channel <= c_max)
    return mask.astype(int)


def create_mask(color, blur, dev):
    b, g, r = color[0], color[1], color[2]
    b_min, b_max = b-dev, b+dev
    g_min, g_max = g-dev, g+dev
    r_min, r_max = r-dev, r+dev
    b_mask, g_mask, r_mask = get_binary_mask(blur[:, :, 0], blur.shape, b_min, b_max), get_binary_mask(
        blur[:, :, 1], blur.shape, g_min, g_max), get_binary_mask(blur[:, :, 2], blur.shape, r_min, r_max)
    sum_mask = b_mask+g_mask+r_mask
    return (sum_mask == 3).astype(int)


In [7]:
def sky_enhancement(img):
    blur = cv2.bilateralFilter(img, 9, 75, 75)
    # Get swatch for sky and mask
    blue = get_swatch(img, "Sky Swatch")
    dominant_blue = dominant_color(blue)
    sky_m = create_mask(dominant_blue, blur, 70)

    # Get swatch for cloud and mask
    cloud = get_swatch(img, "Cloud Swatch")
    dominant_cloud = dominant_color(cloud)
    cloud_m = create_mask(dominant_cloud, blur, 50)

    # join both masks to get full sky mask
    joint_mask = cv2.bitwise_or(sky_m, cloud_m)

    # Get average color and f value
    # Ideal blue is a selected color by the user
    avg_blue = np.zeros(blur.shape, np.uint8)
    avg_blue[:] = (dominant_blue[0], dominant_blue[1], dominant_blue[2])
    ideal_blue = np.zeros(blur.shape, np.uint8)
    ideal_blue[:] = (189, 122, 42)
    cv2.imshow("sky color", np.hstack([ideal_blue, avg_blue]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    avg_blue = cv2.cvtColor(avg_blue, cv2.COLOR_BGR2Lab)
    ideal_blue = cv2.cvtColor(ideal_blue, cv2.COLOR_BGR2Lab)

    # Implement new color in the sky
    f_sky = (ideal_blue/avg_blue)[0, 0]
    # copy where we'll assign the new values
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    sky_enhance = np.copy(lab)
    # boolean indexing and assignment based on mask
    for i in range(0, 3):
        sky_enhance[:, :, i][joint_mask == 1] = sky_enhance[:,
                                                            :, i][joint_mask == 1] * f_sky[i]
    sky_enhance_w = cv2.addWeighted(sky_enhance, 1, img, 0, 0, sky_enhance)
    check = cv2.cvtColor(sky_enhance_w, cv2.COLOR_Lab2BGR)
    cv2.imshow("sky_mask", cv2.bitwise_and(
        img, img, mask=joint_mask.astype(np.uint8)))
    cv2.imshow("new sky", np.hstack([img, check, sky_enhance_w]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Implement new color in the clouds
    # Not getting good results, applies blue instead of white
    # Better to keep blueish clouds from applying new color to whole image
    """
    ideal_white = [100,0,0]
    # copy where we'll assign the new values
    cloud_enhance = np.copy(sky_enhance)
    # boolean indexing and assignment based on mask
    for i in range(0,3):
        cloud_enhance[:,:,i][cloud_m == 1] = (cloud_enhance[:,:,i][cloud_m == 1] + ideal_white[i])/2.0
    
    cloud_enhance[:,:,0].clip(0,100)
    cloud_enhance_w = cv2.addWeighted(cloud_enhance, 1, img, 0, 0, cloud_enhance)
    check_c = cv2.cvtColor(cloud_enhance_w, cv2.COLOR_Lab2BGR)
    cv2.imshow("cloud_mask",cv2.bitwise_and(img, img, mask = cloud_m.astype(np.uint8)))
    cv2.imshow("enhancements sky",np.hstack([img, check_c, cloud_enhance_w]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    """

    """
    βnew = Psky*f sky* βold + (1 − Psky) * βold
    
    κnew = Psky*(W+κold)/2 + (1 − Psky)*  κold
    """

    return check.copy()


In [8]:
colored_img = sky_enhancement(image)
cv2.imshow("Original", image)
cv2.imshow("Final", colored_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
